## Imports

In [1]:
import tensorflow as tf
from tensorflow.keras.models import Model

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
# from pandas_profiling import ProfileReport
%matplotlib inline

## Preprocessing the datasets

### Training Dataset

In [2]:
# train_path = '../input/data-storm-30/train_data.csv'
train_path = 'weekly_train_data.csv'
# train_path = 'train_data.csv'
train_dataset = pd.read_csv(train_path)
train_dataset = train_dataset.drop_duplicates(keep = 'first')
train_dataset = train_dataset.filter(['ItemCode', 'CategoryCode', 'Week_num', 'WeeklySales','Unique_ID'])

In [3]:
print(train_dataset.columns)
print('No. of different categories: {}'.format(len(train_dataset['CategoryCode'].unique())))
print('No. of different items: {}'.format(len(train_dataset['ItemCode'].unique())))


Index(['ItemCode', 'CategoryCode', 'Week_num', 'WeeklySales', 'Unique_ID'], dtype='object')
No. of different categories: 4
No. of different items: 194


In [4]:
# # converting the dtype of the DateID
# train_dataset['DateID'] = pd.to_datetime(train_dataset['DateID'])
# train_dataset.sort_values(by = 'DateID', inplace=True)
# print(train_dataset.dtypes)

In [5]:
train_dataset.head()
# train_dataset

,ItemCode,CategoryCode,Week_num,WeeklySales,Unique_ID
0,3418,category_1,1,29,3418_1
1,3418,category_1,2,42,3418_2
2,3418,category_1,3,41,3418_3
3,3418,category_1,4,41,3418_4
4,3418,category_1,5,44,3418_5


In [6]:
flag = 0
for item in train_dataset['ItemCode'].unique():
    tmp_dataset = train_dataset.loc[train_dataset['ItemCode'] == item]
    if len(tmp_dataset['CategoryCode'].unique()) == 1:
        continue
    else:
        print('CategoryCode for an ItemCode is not unique')
        flag = 1
        break

### Validation Dataset

In [7]:
valid_path = 'weekly_valid_data.csv'
# valid_path = 'validation_data.csv'
validation_dataset = pd.read_csv(valid_path)
validation_dataset = validation_dataset.drop_duplicates(keep = 'first')
validation_dataset = validation_dataset.filter(['ItemCode', 'CategoryCode', 'Week_num', 'WeeklySales','Unique_ID'])

In [8]:
print(validation_dataset.columns)
print('No. of different categories: {}'.format(len(validation_dataset['CategoryCode'].unique())))
print('No. of different items: {}'.format(len(validation_dataset['ItemCode'].unique())))

Index(['ItemCode', 'CategoryCode', 'Week_num', 'WeeklySales'], dtype='object')
No. of different categories: 4
No. of different items: 95


### Test Dataset

In [9]:
test_path = 'test_data.csv'
test_dataset = pd.read_csv(test_path)
test_dataset = test_dataset.drop_duplicates(keep = 'first')

In [10]:
# print(test_dataset.columns)
# print('No. of different categories: {}'.format(len(test_dataset['CategoryCode'].unique())))
# print('No. of different items: {}'.format(len(test_dataset['ItemCode'].unique())))
# test_dataset.shape[0]

## Defining the model


### Windowing function

In [11]:
#if you are using convolutions, expand dim within the helper function
def windowed_dataset(series, window_size, batch_size, shuffle_buffer):
    series = tf.expand_dims(series, axis = -1)
    dataset = tf.data.Dataset.from_tensor_slices(series)
    dataset = dataset.window(window_size +1, shift = 1, drop_remainder = True)
    dataset = dataset.flat_map(lambda window: window.batch(window_size+1))
    dataset = dataset.shuffle(shuffle_buffer).map(lambda window: (window[:-1], window[-1:]))
    dataset = dataset.batch(batch_size).prefetch(1)
    return dataset

### Plot Function

In [12]:
def plot_series(time, series, format="-", start=0, end=None):
    plt.plot(time[start:end], series[start:end], format)
    plt.xlabel("Time")
    plt.ylabel("Value")
    plt.grid(True)

### Forecast Function

In [13]:
#lets define a function for the forecasting part that would be used after training
def model_forecast(model, series, window_size):
    dataset = tf.data.Dataset.from_tensor_slices(series)
    dataset = dataset.window(window_size, shift= 1, drop_remainder = True)
    dataset = dataset.flat_map(lambda window: window.batch(window_size))
    dataset = dataset.batch(32).prefetch(1)
    forecast = model.predict(dataset)
    
    return forecast

### Writing to Test_data

In [14]:
# def edit_dataset(test_dataset,itemCode,predict_list):
#     for i in range(len(predict_list)):
#         week = "w" + str(i+1)
#         x =set(test_dataset[test_dataset['ItemCode']==itemCode].index.values)
#         y =set(test_dataset[test_dataset['Week']==week].index.values)
#         index = list(x.intersection(y))[0]
#         test_dataset.at[index, 'PredictedSales'] = predict_list[i]


### Model

In [15]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Conv1D(filters=60, kernel_size=5,
                      strides=1, padding="causal",
                      activation="relu",
                      input_shape=[None, 1]),
  tf.keras.layers.LSTM(60, return_sequences=True),
  tf.keras.layers.LSTM(60, return_sequences=True),
  tf.keras.layers.Dense(30, activation="relu"),
  tf.keras.layers.Dense(10, activation="relu"),
  tf.keras.layers.Dense(1),
  tf.keras.layers.Lambda(lambda x: x * 50)
])


optimizer = tf.keras.optimizers.SGD(lr=1e-5, momentum=0.9)
loss = tf.keras.losses.Huber()

model.compile(loss= loss,optimizer=optimizer,metrics=[tf.keras.metrics.MeanAbsolutePercentageError()])


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


### Creating an empty dataframe

In [16]:
result_df = pd.DataFrame(columns = ('CategoryCode', 'ItemCode','Week', 'PredictedSales'))

In [17]:
def write_to_df(df, category_code, item_code, predicted_sales):
  for i in range(len(predicted_sales)):
    df = df.append({'CategoryCode':category_code,'ItemCode':item_code, 'Week': 'w'+str(i+1), 'PredictedSales':predicted_sales[i]}, ignore_index = True)
  return df

### Picking an item

In [ ]:
for item_num in test_dataset['ItemCode'].unique():
  tmp_dataset_train = train_dataset.loc[train_dataset['ItemCode'] == item_num]
  weeks_train = np.array(tmp_dataset_train['Week_num'])
  sales_train = np.array(tmp_dataset_train['WeeklySales'])

  offset = len(weeks_train)

  # tmp_dataset_valid = validation_dataset.loc[validation_dataset['ItemCode'] == item_num]
  # weeks_valid = np.array(tmp_dataset_valid.pop('Week_num'))
  # weeks_valid = [offset + i for i in weeks_valid]
  # sales_valid = np.array(tmp_dataset_valid['WeeklySales'])

  weeks_test  = np.array([offset + i for i in range(4)])
  sales_test = np.zeros((4,))

  total_weeks = np.concatenate([weeks_train, weeks_test])
  total_sales = np.concatenate([sales_train, sales_test])

  window_size = 5
  batch_size = 10
  shuffle_buffer_size = 15
  train_set = windowed_dataset(sales_train, window_size, batch_size, shuffle_buffer_size)
  
  tf.keras.backend.clear_session()
  tf.random.set_seed(51)
  np.random.seed(51)

  history = model.fit(train_set,epochs=500)

  lstm_forecast = model_forecast(model, total_sales[..., np.newaxis], window_size)
  lstm_forecast = lstm_forecast[offset - window_size:-1, -1, 0]

  # tf.keras.metrics.mean_absolute_percentage_error(sales_valid, lstm_forecast).numpy()
  category = tmp_dataset_train['CategoryCode'].unique()[0]
  result_df = write_to_df(result_df, category, item_num, lstm_forecast)

Streaming output truncated to the last 5000 lines.
2/2 [==============================] - 0s 16ms/step - loss: 9.4536 - mean_absolute_percentage_error: 12.7646
Epoch 61/500
2/2 [==============================] - 0s 16ms/step - loss: 9.3168 - mean_absolute_percentage_error: 12.6628
Epoch 62/500
2/2 [==============================] - 0s 15ms/step - loss: 9.2786 - mean_absolute_percentage_error: 12.7746
Epoch 63/500
2/2 [==============================] - 0s 19ms/step - loss: 9.3806 - mean_absolute_percentage_error: 12.9938
Epoch 64/500
2/2 [==============================] - 0s 23ms/step - loss: 9.3961 - mean_absolute_percentage_error: 13.0464
Epoch 65/500
2/2 [==============================] - 0s 18ms/step - loss: 9.2964 - mean_absolute_percentage_error: 12.8230
Epoch 66/500
2/2 [==============================] - 0s 16ms/step - loss: 9.2097 - mean_absolute_percentage_error: 12.6025
Epoch 67/500
2/2 [==============================] - 0s 15ms/step - loss: 9.3088 - mean_absolute_percentage_e

In [ ]:
result_df.to_csv('lstm_test_results.csv', index=False)